# 📶 Meshtastic Buddy List

### *A script for checking which Meshtastic nodes are currently active*

You'll need a LoRa radio with Meshtastic installed. If you're just getting started, check out our how-to zine:
https://iffybooks.net/meshtastic-zine-2023

Connect your LoRa radio to your computer's USB port before running this script.

In [ ]:
## Install dependencies

# Uncomment the command below to install the Meshtastic CLI
#!python3.9 -m pip install meshtastic

In [ ]:
import os
from datetime import datetime
import time

# Change directory to the desktop
os.chdir(os.path.expanduser("~/Desktop/"))

In [ ]:
# Delete previous data file before starting (feel free to comment this out)
if "meshtastic_data.txt" in os.listdir(): os.remove("meshtastic_data.txt")

# Command for storing the current time and the output of `meshtastic --nodes` to `meshtastic_data.txt`
command = '''
echo $(date -I seconds) >> meshtastic_data.txt
meshtastic --nodes >> meshtastic_data.txt
echo "\n\n" >> meshtastic_data.txt
'''

# Run the command once per minute for 15 minutes
for i in range(15):
    print(str(datetime.now()))
    os.system(command)
    time.sleep(60)

In [ ]:
# Read data file and split into chunks

with open("/Users/iffybooks/Desktop/meshtastic_data.txt") as fi:
    plaintext = fi.read()

text_chunks = [item.strip() for item in plaintext.split("\n\n") if item.strip()!='']
len(text_chunks)

In [ ]:
# Create a dictionary with node names and a dictionary with timestamps as datetime objects

node_dict = {}
name_dict = {}

for text_chunk in text_chunks:
    lines = text_chunk.split('\n')
    
    # Store real timestamp string
    real_timestamp = lines[0]
    #print(lines)
    #print()
    
    # Extract real timestamp metadata
    year = int(real_timestamp.split('-')[0])
    month = int(real_timestamp.split('-')[1])
    day = int(real_timestamp.split('T')[0].split('-')[2])
    hour = int(real_timestamp.split('T')[1].split(':')[0])
    minute = int(real_timestamp.split('T')[1].split(':')[1])
    second = int(real_timestamp.split('T')[1].split(':')[2].split('-')[0])
    
    # Create a datetime object
    real_datetime = datetime(year, month, day, hour, minute, second)

    # Remove timestamp line
    lines = lines[1:]
    
    # Remove lines we dont need
    lines = [item.strip() for item in lines if '══════════════' not in item]
    lines = [item.strip() for item in lines if '────────' not in item]
    lines = [item for item in lines if item.strip()!='']
    lines = [item for item in lines if 'Warning: Multiple serial ports' not in item]
    lines = [item for item in lines if 'Ports detected:' not in item]
    lines = [item for item in lines if item.strip()!='Connected to radio']
    
    # Convert node data to a list of lists
    lol = []
    for line in lines:
        row = line.split('│')
        lol.append(row)

    # Remove header row
    header = lol[0]
    lol = lol[1:]
    
    for row in lol:
        
        # Extract metadata from node timestamp
        name = row[2].strip()
        node_id = row[4].strip()
        name_dict[node_id] = name        
        wrong_timestamp = row[12].strip()
        
        try: year = int(wrong_timestamp.split(' ')[0].split('-')[0])
        except: year = 1900

        try: month = int(wrong_timestamp.split(' ')[0].split('-')[1])
        except: month = 1
        
        try: day = int(wrong_timestamp.split(' ')[0].split('-')[2])
        except: day = 1
            
        try: hour = int(wrong_timestamp.split(' ')[1].split(':')[0])
        except: hour = 0
        
        try: minute = int(wrong_timestamp.split(' ')[1].split(':')[1])
        except: minute = 0
        
        try: second = int(wrong_timestamp.split(' ')[1].split(':')[2])
        except: second = 0
        
        # Create a datetime object
        wrong_datetime = datetime(year, month, day, hour, minute, second)

        # Create list of datetimes
        datetime_list = [real_datetime, wrong_datetime]
        
        # Add datetime list to node dict
        if node_id not in node_dict:
            node_dict[node_id] = [datetime_list]
        else:
            node_dict[node_id].append(datetime_list)
            
        
    
len(node_dict)

In [ ]:
# For each node we'll check whether the "last updated" time has changed. If it has, we'll consider the node active.

# We'll store active node IDs as a set
active_node_set = set()

for node_id in node_dict:
    pair_list = node_dict[node_id]
    
    print(name_dict[node_id])
    print(node_id)

    # Store and remove the first pair of datetime objects
    previous_real_datetime, previous_wrong_datetime = pair_list[0]
    pair_list = pair_list[1:]

    for real_datetime, wrong_datetime in pair_list:
        
        # Calculate and print the time difference between this observation and the previous one
        time_difference_string = str(wrong_datetime - previous_wrong_datetime)
        print(time_difference_string)
        
        # If there's been any change, we'll consider the node active. (This part needs work.)
        if time_difference_string != "0:00:00":
            active_node_set.add(node_id)

        # Update previous datetime variables
        previous_real_datetime = real_datetime
        previous_wrong_datetime = wrong_datetime
    
    # Print a blank line
    print()

In [ ]:
## Print a list of active nodes

print("Active nodes:\n")

for node_id in sorted(list(active_node_set)):
    
    print(name_dict[node_id])
    print(node_id)
    print()